Based on **Francesco Pierfederici: Distributed Computing with Python, Chapter 2**

# An asynchronous example

Things become interesting when we start **organizing coroutines into complex  graphs**. 

For instance, we might want to **count the concurrence of multiple words in the input file**.

The below code is an example for this:

In [1]:
# We will use this as a decorator for our coroutines
def coroutine(fn):
    def wrapper(*args, **kwargs):
        c = fn(*args, **kwargs)
        next(c)
        return c
    return wrapper


In [2]:
# These are the similar to the functions in the previous example
def cat(f, child):
    for line in f:
        child.send(line)

@coroutine
def grep(substring, child):
    try:
        substring = substring.lower()
        while True:
            text = (yield)
            child.send(text.count(substring))
    except GeneratorExit:
        print('exit grep')
        
@coroutine
def count(substring):
    n = 0
    try:
        while True:
            tmp = (yield)
            n += tmp
            if tmp > 0:
                print(substring, n)
    except GeneratorExit:
        print('exit count')
        print(substring, n)


In [4]:
@coroutine
def fanout(children):
    try: 
        while True:
            data = (yield) #this will get a line
            for child in children: #Here we will send the data (=line) to multiple coroutines (grep function) listed in the children variable
                child.send(data)
    except GeneratorExit:
        print('exit fanout')

In [5]:
cat(f=list(open('../data/pg2600.txt'))[0:5000], child=fanout(children=[grep(substring=p, child=count(p)) for p in ['love','hate','hope']]))

hope 1
hope 2
love 1
love 2
hate 1
love 3
hope 3
hope 4
love 4
hope 6
love 5
hope 7
hope 8
love 6
love 7
love 8
love 9
love 10
love 11
love 12
love 13
hope 9
hope 10
hope 11
love 14
hope 12
hope 13
hope 14
hate 2
love 15
love 16
love 17
love 18
love 19
hope 15
love 20
love 21
love 22
hope 16
love 23
love 24
love 25
exit fanout
exit grep
exit count
hate 2
exit grep
exit count
love 25
exit grep
exit count
hope 16


### Other tools:
Python 3.4 introduced a **new library for asynchronous I/O** called **asyncio**.

Python 3.5 introduced **coroutine types** via **async def** and **await**.

